In [1]:
import skimage
from skimage import transform
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from scipy.fftpack import fft2, fftshift, ifft
from skimage import img_as_float
from skimage.color import rgb2gray
from skimage import io
import skimage
%matplotlib qt
sys.path.append("..")
from crystalmaths import imagetools
import crystalmaths

In [2]:
image_directory = r'C:\Users\lacho\OneDrive - UW\crystalmaths\Local_data'
image_list = []
for root, dirs, files in os.walk(image_directory):
    if root == image_directory:
        for file in files:
            image_list.append(root+os.sep+file)

print(image_list)

['C:\\Users\\lacho\\OneDrive - UW\\crystalmaths\\Local_data\\Cubic_1.bmp']


In [ ]:
image_object = imagetools.ImageHandler(image_list[0], window_type='hann')
image_object.get_scale_ratio()
# image_object.show_image()
image_object.get_planes()
# image_object.(pixel_to_nm_ratio)
# image_object.get_scale() #Needs prompt for scale bar in nm
# image_object.pixel_to_nm_ratio

Enter scalebar length in nanometers.2


In [ ]:
image_object.scale_ratio

In [ ]:
image_object.image_array.shape

In [ ]:
def get_image(image_path):
    image = skimage.io.imread(image_path)
#     image2 = skimage.img_as_float(image_path)
    return image

def get_fft(image_array):
    image_fft = np.abs(np.fft.fftshift((np.fft.fft2(image_array))))
    image_fft = np.log(image_fft)
    return image_fft

def crop_image(image_array):
    """
    Obtain an inner square crop, assuming that this eliminates scale bar. 
    Assumption: scale bars aren't present in inner 80% of image.
    """
    x, y= image_array.shape[0], image_array.shape[1]
    
    #Start of 20% of image in x dimension
    percent_crop = 0.2
    x_0 = int(x*(percent_crop))
    x_f = int(x*(1-percent_crop))
    y_0 = int(y*(percent_crop))
    y_f = int(y*(1-percent_crop))
    
    cropped_image = image_array[x_0:x_f,y_0:y_f]
    return cropped_image

def window_image(image_array, window_name):
    filter_options = {"hann":apply_hann(image_array)}
    
    filtered_image = filter_options[window_name]
    
    return filtered_image

def apply_hann(image_array):
    
    return

## Q. Why did cropping the scale bar out not remove the streak? 


A: Axial streaking is the result of discontinuities at border? 

Resources:
https://dsp.stackexchange.com/questions/25927/fourier-transform-artifacts 
**I am interested in trying the periodic plus smoth image decomposition to get rid of artifacts, it has already been applied to TEM images before, see pub: https://www.cambridge.org/core/journals/microscopy-and-microanalysis/article/periodic-artifact-reduction-in-fourier-transforms-of-full-field-atomic-resolution-images/80D0E226F0B4B16627AA0B6B9BD24F24, see demo: https://github.com/jacobkimmel/ps_decomp**

Also consider:
https://dsp.stackexchange.com/questions/26830/python-scipy-fft-on-numpy-hanning-window-smears-peaks

#Code_Inspo:
https://github.com/wonderchang/img-fft/blob/master/img-fft.py



In [ ]:
from scipy import ndimage as ndi
import matplotlib.pyplot as plt
from skimage.feature import peak_local_max
from skimage import data, img_as_float

im = img_as_float(image_object.image_fft)

# image_max is the dilation of im with a 20*20 structuring element
# It is used within peak_local_max function
image_max = ndi.maximum_filter(im, size=20, mode='constant')

# Comparison between image_max and im to find the coordinates of local maxima
coordinates = peak_local_max(im, min_distance=20)

# display results
fig, axes = plt.subplots(1, 3, figsize=(8, 3), sharex=True, sharey=True)
ax = axes.ravel()
ax[0].imshow(im, cmap=plt.cm.gray)
ax[0].axis('off')
ax[0].set_title('Original')

ax[1].imshow(image_max, cmap=plt.cm.gray)
ax[1].axis('off')
ax[1].set_title('Maximum filter')

ax[2].imshow(im, cmap=plt.cm.gray)
ax[2].autoscale(False)
ax[2].plot(coordinates[:, 1], coordinates[:, 0], 'r.')
ax[2].axis('off')
ax[2].set_title('Peak local max')

fig.tight_layout()

plt.show()
    